# RSL (Relative Stärke Levy) Stock Screening Strategy

## Overview
This notebook implements the **Relative Strength Levy (RSL)** strategy to screen and rank S&P 500 stocks.

### What is RSL?
The Relative Strength Levy indicator was developed by Robert Levy in 1967. It measures a stock's momentum by comparing its current price to its historical average.

**Formula:**
```
RSL = Current Price / SMA(Price, N periods)
```

Where:
- **Current Price** = Latest closing price
- **SMA** = Simple Moving Average over N periods
- **N** = Typically 27 weeks (~130 trading days) or 52 weeks (~260 trading days)

### Interpretation:
- **RSL > 1.0**: Stock is trading above its average (bullish momentum)
- **RSL < 1.0**: Stock is trading below its average (bearish momentum)
- **Higher RSL**: Stronger relative strength

### Strategy:
Buy stocks in the top 25% by RSL ranking (strongest momentum stocks).

---

## 1. Installation
Install required libraries (run this cell first in Google Colab)

In [ ]:
# Install required packages
!pip install yfinance pandas openpyxl beautifulsoup4 lxml tqdm xlsxwriter --quiet
print("All packages installed successfully!")

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm
import time
import warnings
import os

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', None)

print("Libraries imported successfully!")
print(f"Run Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 3. Configuration Parameters
Adjust these parameters to customize the analysis

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# RSL Calculation Period (in trading days)
# 27 weeks = ~130 trading days (Levy's original)
# 52 weeks = ~260 trading days (1 year)
RSL_PERIOD = 130  # 27 weeks / ~6 months

# Data fetch period (need extra days for SMA calculation)
LOOKBACK_DAYS = 365  # 1 year of data to ensure enough history

# Rate limiting (seconds between API calls)
API_DELAY = 0.1  # 100ms delay between requests

# Top performers threshold
TOP_PERCENTILE = 0.25  # Top 25%

# Output file name
OUTPUT_FILE = f"RSL_Rankings_{datetime.now().strftime('%Y%m%d')}.xlsx"

print("Configuration loaded:")
print(f"  - RSL Period: {RSL_PERIOD} trading days (~{RSL_PERIOD//5} weeks)")
print(f"  - Lookback Period: {LOOKBACK_DAYS} days")
print(f"  - Top Performers: Top {int(TOP_PERCENTILE*100)}%")
print(f"  - Output File: {OUTPUT_FILE}")

## 4. Fetch S&P 500 Tickers from Wikipedia
Scrape the current list of S&P 500 companies

In [ ]:
def fetch_sp500_tickers():
    """
    Fetch S&P 500 company tickers and names from Wikipedia.
    
    Returns:
        DataFrame with columns: Symbol, Company, Sector, Industry
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    
    print("Fetching S&P 500 tickers from Wikipedia...")
    
    try:
        # Use requests with headers to avoid blocking
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.text, 'lxml')
        table = soup.find('table', {'id': 'constituents'})
        
        if table is None:
            # Fallback: find first table with expected structure
            tables = soup.find_all('table', {'class': 'wikitable'})
            for t in tables:
                if t.find('th', string=lambda x: x and 'Symbol' in x):
                    table = t
                    break
        
        # Read table into DataFrame
        df = pd.read_html(str(table))[0]
        
        # Standardize column names
        df.columns = df.columns.str.strip()
        
        # Extract relevant columns
        result = pd.DataFrame({
            'Symbol': df['Symbol'].str.strip().str.replace('.', '-', regex=False),  # Yahoo uses - instead of .
            'Company': df['Security'].str.strip(),
            'Sector': df['GICS Sector'].str.strip() if 'GICS Sector' in df.columns else 'N/A',
            'Industry': df['GICS Sub-Industry'].str.strip() if 'GICS Sub-Industry' in df.columns else 'N/A'
        })
        
        print(f"Successfully fetched {len(result)} S&P 500 tickers!")
        return result
        
    except Exception as e:
        print(f"Error fetching S&P 500 list: {e}")
        print("Using fallback ticker list...")
        
        # Fallback: minimal list for testing
        fallback_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 
                          'BRK-B', 'UNH', 'JNJ', 'V', 'XOM', 'JPM', 'PG', 'MA']
        return pd.DataFrame({
            'Symbol': fallback_tickers,
            'Company': ['Apple Inc.', 'Microsoft Corp.', 'Alphabet Inc.', 'Amazon.com Inc.',
                       'NVIDIA Corp.', 'Meta Platforms', 'Tesla Inc.', 'Berkshire Hathaway',
                       'UnitedHealth Group', 'Johnson & Johnson', 'Visa Inc.', 'Exxon Mobil',
                       'JPMorgan Chase', 'Procter & Gamble', 'Mastercard Inc.'],
            'Sector': ['Technology'] * 15,
            'Industry': ['Various'] * 15
        })

# Fetch the tickers
sp500_df = fetch_sp500_tickers()
print(f"\nFirst 10 tickers:")
print(sp500_df.head(10).to_string(index=False))

## 5. Define RSL Calculation Functions

In [ ]:
def calculate_rsl(prices, period=130):
    """
    Calculate Relative Strength Levy (RSL) for a price series.
    
    RSL = Current Price / SMA(Price, period)
    
    Args:
        prices: Series of closing prices
        period: Number of periods for SMA calculation
    
    Returns:
        float: RSL value, or None if calculation fails
    """
    if prices is None or len(prices) < period:
        return None
    
    try:
        # Get current price (latest)
        current_price = prices.iloc[-1]
        
        # Calculate Simple Moving Average
        sma = prices.iloc[-period:].mean()
        
        # Avoid division by zero
        if sma == 0 or pd.isna(sma):
            return None
        
        # Calculate RSL
        rsl = current_price / sma
        
        return round(rsl, 4)
        
    except Exception:
        return None


def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetch historical stock data from Yahoo Finance.
    
    Args:
        ticker: Stock ticker symbol
        start_date: Start date for data fetch
        end_date: End date for data fetch
    
    Returns:
        dict with price data and metadata, or None if fetch fails
    """
    try:
        # Fetch data
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date, auto_adjust=True)
        
        if hist.empty or len(hist) < RSL_PERIOD:
            return None
        
        # Get closing prices
        close_prices = hist['Close']
        
        # Calculate RSL
        rsl = calculate_rsl(close_prices, RSL_PERIOD)
        
        if rsl is None:
            return None
        
        # Get additional metrics
        current_price = close_prices.iloc[-1]
        price_52w_high = close_prices.max()
        price_52w_low = close_prices.min()
        
        # Calculate percentage from 52-week high
        pct_from_high = ((current_price - price_52w_high) / price_52w_high) * 100
        
        # Calculate price change percentages
        if len(close_prices) >= 20:
            price_1m_ago = close_prices.iloc[-20] if len(close_prices) >= 20 else close_prices.iloc[0]
            change_1m = ((current_price - price_1m_ago) / price_1m_ago) * 100
        else:
            change_1m = 0
            
        if len(close_prices) >= 60:
            price_3m_ago = close_prices.iloc[-60]
            change_3m = ((current_price - price_3m_ago) / price_3m_ago) * 100
        else:
            change_3m = 0
            
        if len(close_prices) >= 130:
            price_6m_ago = close_prices.iloc[-130]
            change_6m = ((current_price - price_6m_ago) / price_6m_ago) * 100
        else:
            change_6m = 0
        
        return {
            'RSL': rsl,
            'Current_Price': round(current_price, 2),
            '52W_High': round(price_52w_high, 2),
            '52W_Low': round(price_52w_low, 2),
            'Pct_From_High': round(pct_from_high, 2),
            'Change_1M': round(change_1m, 2),
            'Change_3M': round(change_3m, 2),
            'Change_6M': round(change_6m, 2),
            'Data_Points': len(close_prices)
        }
        
    except Exception as e:
        return None

print("RSL calculation functions defined successfully!")

## 6. Fetch Data and Calculate RSL for All Stocks
This may take several minutes due to API rate limiting

In [ ]:
def process_all_stocks(tickers_df, start_date, end_date):
    """
    Process all stocks: fetch data and calculate RSL.
    
    Args:
        tickers_df: DataFrame with Symbol, Company, Sector, Industry columns
        start_date: Start date for data fetch
        end_date: End date for data fetch
    
    Returns:
        DataFrame with RSL rankings
    """
    results = []
    failed_tickers = []
    
    print(f"\nProcessing {len(tickers_df)} stocks...")
    print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    print("-" * 60)
    
    for idx, row in tqdm(tickers_df.iterrows(), total=len(tickers_df), desc="Fetching data"):
        ticker = row['Symbol']
        company = row['Company']
        sector = row['Sector']
        industry = row['Industry']
        
        # Fetch and calculate
        data = fetch_stock_data(ticker, start_date, end_date)
        
        if data:
            results.append({
                'Ticker': ticker,
                'Company': company,
                'Sector': sector,
                'Industry': industry,
                **data
            })
        else:
            failed_tickers.append(ticker)
        
        # Rate limiting
        time.sleep(API_DELAY)
    
    print(f"\n" + "=" * 60)
    print(f"Processing complete!")
    print(f"  - Successfully processed: {len(results)} stocks")
    print(f"  - Failed/Skipped: {len(failed_tickers)} stocks")
    
    if failed_tickers and len(failed_tickers) <= 20:
        print(f"  - Failed tickers: {', '.join(failed_tickers)}")
    elif failed_tickers:
        print(f"  - First 20 failed: {', '.join(failed_tickers[:20])}...")
    
    # Create DataFrame and sort by RSL
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('RSL', ascending=False).reset_index(drop=True)
    
    # Add rank column
    results_df.insert(0, 'Rank', range(1, len(results_df) + 1))
    
    return results_df, failed_tickers


# Calculate date range
end_date = datetime.now()
start_date = end_date - timedelta(days=LOOKBACK_DAYS)

# Process all stocks
results_df, failed_tickers = process_all_stocks(sp500_df, start_date, end_date)

print(f"\nRSL calculation complete for {len(results_df)} stocks!")

## 7. View Results and Summary Statistics

In [ ]:
# Display summary statistics
print("=" * 70)
print("RSL SUMMARY STATISTICS")
print("=" * 70)
print(f"\nTotal stocks analyzed: {len(results_df)}")
print(f"\nRSL Statistics:")
print(f"  - Mean RSL:   {results_df['RSL'].mean():.4f}")
print(f"  - Median RSL: {results_df['RSL'].median():.4f}")
print(f"  - Max RSL:    {results_df['RSL'].max():.4f} ({results_df.loc[results_df['RSL'].idxmax(), 'Ticker']})")
print(f"  - Min RSL:    {results_df['RSL'].min():.4f} ({results_df.loc[results_df['RSL'].idxmin(), 'Ticker']})")
print(f"  - Std Dev:    {results_df['RSL'].std():.4f}")

# Count stocks above/below 1.0
above_one = (results_df['RSL'] > 1.0).sum()
below_one = (results_df['RSL'] < 1.0).sum()
print(f"\nMomentum Distribution:")
print(f"  - RSL > 1.0 (bullish): {above_one} stocks ({above_one/len(results_df)*100:.1f}%)")
print(f"  - RSL < 1.0 (bearish): {below_one} stocks ({below_one/len(results_df)*100:.1f}%)")

# Percentile thresholds
print(f"\nPercentile Thresholds:")
for pct in [25, 50, 75, 90, 95]:
    threshold = results_df['RSL'].quantile(pct/100)
    print(f"  - {pct}th percentile: RSL >= {threshold:.4f}")

In [ ]:
# Display Top 25 stocks
print("\n" + "=" * 70)
print("TOP 25 STOCKS BY RSL (Strongest Momentum)")
print("=" * 70 + "\n")

top_25_display = results_df.head(25)[['Rank', 'Ticker', 'Company', 'RSL', 'Current_Price', 
                                       'Change_1M', 'Change_3M', 'Change_6M', 'Pct_From_High']]
print(top_25_display.to_string(index=False))

In [ ]:
# Display Bottom 10 stocks
print("\n" + "=" * 70)
print("BOTTOM 10 STOCKS BY RSL (Weakest Momentum)")
print("=" * 70 + "\n")

bottom_10_display = results_df.tail(10)[['Rank', 'Ticker', 'Company', 'RSL', 'Current_Price', 
                                          'Change_1M', 'Change_3M', 'Change_6M', 'Pct_From_High']]
print(bottom_10_display.to_string(index=False))

In [ ]:
# Sector breakdown
print("\n" + "=" * 70)
print("SECTOR ANALYSIS - Average RSL by Sector")
print("=" * 70 + "\n")

sector_stats = results_df.groupby('Sector').agg({
    'RSL': ['mean', 'median', 'count'],
    'Change_6M': 'mean'
}).round(4)

sector_stats.columns = ['Avg_RSL', 'Median_RSL', 'Stock_Count', 'Avg_6M_Change']
sector_stats = sector_stats.sort_values('Avg_RSL', ascending=False)

print(sector_stats.to_string())

## 8. Create Top 25% Portfolio (Buy Candidates)

In [ ]:
# Calculate top 25% threshold
top_threshold = int(len(results_df) * TOP_PERCENTILE)
rsl_threshold = results_df.iloc[top_threshold - 1]['RSL'] if top_threshold > 0 else results_df['RSL'].max()

# Create top performers DataFrame
top_performers = results_df.head(top_threshold).copy()
top_performers['Percentile'] = 'Top 25%'

print("=" * 70)
print(f"TOP {int(TOP_PERCENTILE*100)}% PORTFOLIO - BUY CANDIDATES")
print("=" * 70)
print(f"\nNumber of stocks: {len(top_performers)}")
print(f"RSL threshold: >= {rsl_threshold:.4f}")
print(f"\nPortfolio Statistics:")
print(f"  - Average RSL: {top_performers['RSL'].mean():.4f}")
print(f"  - Average 1M Change: {top_performers['Change_1M'].mean():.2f}%")
print(f"  - Average 3M Change: {top_performers['Change_3M'].mean():.2f}%")
print(f"  - Average 6M Change: {top_performers['Change_6M'].mean():.2f}%")

print(f"\nSector Allocation:")
sector_allocation = top_performers['Sector'].value_counts()
for sector, count in sector_allocation.items():
    pct = count / len(top_performers) * 100
    print(f"  - {sector}: {count} stocks ({pct:.1f}%)")

## 9. Generate Excel Report with Multiple Sheets

In [ ]:
def create_excel_report(results_df, top_performers, sector_stats, failed_tickers, output_file):
    """
    Create a professional Excel report with multiple sheets and formatting.
    """
    print(f"\nGenerating Excel report: {output_file}")
    
    # Create Excel writer with xlsxwriter engine for better formatting
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        workbook = writer.book
        
        # Define formats
        header_format = workbook.add_format({
            'bold': True,
            'bg_color': '#1F4E79',
            'font_color': 'white',
            'border': 1,
            'align': 'center',
            'valign': 'vcenter'
        })
        
        number_format = workbook.add_format({'num_format': '#,##0.00', 'border': 1})
        percent_format = workbook.add_format({'num_format': '0.00%', 'border': 1})
        rsl_high_format = workbook.add_format({'num_format': '0.0000', 'bg_color': '#C6EFCE', 'border': 1})
        rsl_low_format = workbook.add_format({'num_format': '0.0000', 'bg_color': '#FFC7CE', 'border': 1})
        rsl_normal_format = workbook.add_format({'num_format': '0.0000', 'border': 1})
        text_format = workbook.add_format({'border': 1})
        
        # =====================================================================
        # Sheet 1: Summary
        # =====================================================================
        summary_data = {
            'Metric': [
                'Report Date',
                'RSL Period (days)',
                'Total Stocks Analyzed',
                'Failed/Skipped Tickers',
                '',
                'Mean RSL',
                'Median RSL',
                'Max RSL',
                'Min RSL',
                'Std Dev RSL',
                '',
                'Stocks with RSL > 1.0',
                'Stocks with RSL < 1.0',
                '',
                'Top 25% Threshold (RSL)',
                'Top 25% Count'
            ],
            'Value': [
                datetime.now().strftime('%Y-%m-%d %H:%M'),
                RSL_PERIOD,
                len(results_df),
                len(failed_tickers),
                '',
                f"{results_df['RSL'].mean():.4f}",
                f"{results_df['RSL'].median():.4f}",
                f"{results_df['RSL'].max():.4f}",
                f"{results_df['RSL'].min():.4f}",
                f"{results_df['RSL'].std():.4f}",
                '',
                f"{(results_df['RSL'] > 1.0).sum()} ({(results_df['RSL'] > 1.0).sum()/len(results_df)*100:.1f}%)",
                f"{(results_df['RSL'] < 1.0).sum()} ({(results_df['RSL'] < 1.0).sum()/len(results_df)*100:.1f}%)",
                '',
                f"{top_performers['RSL'].min():.4f}",
                len(top_performers)
            ]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False, startrow=1)
        
        ws_summary = writer.sheets['Summary']
        ws_summary.write(0, 0, 'RSL SCREENING SUMMARY', workbook.add_format({'bold': True, 'font_size': 14}))
        ws_summary.set_column('A:A', 30)
        ws_summary.set_column('B:B', 25)
        
        # =====================================================================
        # Sheet 2: Full Rankings
        # =====================================================================
        results_df.to_excel(writer, sheet_name='Full Rankings', index=False, startrow=0)
        
        ws_full = writer.sheets['Full Rankings']
        
        # Apply header format
        for col_num, value in enumerate(results_df.columns.values):
            ws_full.write(0, col_num, value, header_format)
        
        # Set column widths
        ws_full.set_column('A:A', 6)   # Rank
        ws_full.set_column('B:B', 8)   # Ticker
        ws_full.set_column('C:C', 30)  # Company
        ws_full.set_column('D:D', 20)  # Sector
        ws_full.set_column('E:E', 25)  # Industry
        ws_full.set_column('F:F', 10)  # RSL
        ws_full.set_column('G:N', 12)  # Other columns
        
        # Add conditional formatting for RSL column (column F, index 5)
        rsl_col = 5
        ws_full.conditional_format(1, rsl_col, len(results_df), rsl_col, {
            'type': 'cell',
            'criteria': '>=',
            'value': 1.05,
            'format': rsl_high_format
        })
        ws_full.conditional_format(1, rsl_col, len(results_df), rsl_col, {
            'type': 'cell',
            'criteria': '<',
            'value': 0.95,
            'format': rsl_low_format
        })
        
        # Freeze header row
        ws_full.freeze_panes(1, 0)
        
        # =====================================================================
        # Sheet 3: Top 25% (Buy Candidates)
        # =====================================================================
        top_performers.to_excel(writer, sheet_name='Top 25% Buy List', index=False, startrow=0)
        
        ws_top = writer.sheets['Top 25% Buy List']
        
        for col_num, value in enumerate(top_performers.columns.values):
            ws_top.write(0, col_num, value, header_format)
        
        ws_top.set_column('A:A', 6)
        ws_top.set_column('B:B', 8)
        ws_top.set_column('C:C', 30)
        ws_top.set_column('D:D', 20)
        ws_top.set_column('E:E', 25)
        ws_top.set_column('F:O', 12)
        ws_top.freeze_panes(1, 0)
        
        # =====================================================================
        # Sheet 4: Sector Analysis
        # =====================================================================
        sector_stats.reset_index().to_excel(writer, sheet_name='Sector Analysis', index=False, startrow=0)
        
        ws_sector = writer.sheets['Sector Analysis']
        ws_sector.set_column('A:A', 25)
        ws_sector.set_column('B:E', 15)
        
        # =====================================================================
        # Sheet 5: Failed Tickers (if any)
        # =====================================================================
        if failed_tickers:
            failed_df = pd.DataFrame({'Failed_Ticker': failed_tickers})
            failed_df.to_excel(writer, sheet_name='Failed Tickers', index=False)
    
    print(f"Excel report generated successfully!")
    print(f"File saved: {output_file}")
    return output_file


# Generate the Excel report
excel_file = create_excel_report(results_df, top_performers, sector_stats, failed_tickers, OUTPUT_FILE)

## 10. Download the Excel File (Google Colab)

In [ ]:
# Download file in Google Colab
try:
    from google.colab import files
    files.download(excel_file)
    print(f"\nDownload started for: {excel_file}")
except ImportError:
    # Not running in Colab
    print(f"\nNot running in Google Colab.")
    print(f"Excel file saved locally: {os.path.abspath(excel_file)}")
except Exception as e:
    print(f"\nCould not initiate download: {e}")
    print(f"Excel file saved at: {os.path.abspath(excel_file)}")

## 11. Optional: Visualization

In [ ]:
# Simple text-based visualization (works without matplotlib)
print("\n" + "=" * 70)
print("RSL DISTRIBUTION HISTOGRAM")
print("=" * 70)

# Create histogram bins
bins = [0.7, 0.8, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5]
labels = ['<0.8', '0.8-0.9', '0.9-0.95', '0.95-1.0', '1.0-1.05', '1.05-1.1', '1.1-1.2', '1.2-1.3', '>1.3']

# Count stocks in each bin
counts = pd.cut(results_df['RSL'], bins=bins, labels=labels[:-1]).value_counts().sort_index()

# Also count outliers
low_outliers = (results_df['RSL'] < 0.7).sum()
high_outliers = (results_df['RSL'] > 1.5).sum()

print("\nRSL Range      | Count | Bar")
print("-" * 50)

if low_outliers > 0:
    bar = '#' * (low_outliers * 2)
    print(f"< 0.7          | {low_outliers:5} | {bar}")

for label, count in counts.items():
    bar = '#' * (count * 2 // 3)  # Scale for display
    print(f"{label:14} | {count:5} | {bar}")

if high_outliers > 0:
    bar = '#' * (high_outliers * 2)
    print(f"> 1.5          | {high_outliers:5} | {bar}")

print("\nLegend: # = ~1.5 stocks")

In [ ]:
# Optional: Matplotlib visualization (uncomment if matplotlib is available)
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # RSL Distribution Histogram
    axes[0].hist(results_df['RSL'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0].axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='RSL = 1.0')
    axes[0].axvline(x=results_df['RSL'].quantile(0.75), color='green', linestyle='--', 
                    linewidth=2, label=f'Top 25% threshold')
    axes[0].set_xlabel('RSL Value', fontsize=12)
    axes[0].set_ylabel('Number of Stocks', fontsize=12)
    axes[0].set_title('RSL Distribution of S&P 500 Stocks', fontsize=14)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Sector Average RSL
    sector_avg = results_df.groupby('Sector')['RSL'].mean().sort_values(ascending=True)
    colors = ['green' if x > 1.0 else 'red' for x in sector_avg.values]
    axes[1].barh(range(len(sector_avg)), sector_avg.values, color=colors, alpha=0.7)
    axes[1].set_yticks(range(len(sector_avg)))
    axes[1].set_yticklabels(sector_avg.index, fontsize=10)
    axes[1].axvline(x=1.0, color='black', linestyle='--', linewidth=2)
    axes[1].set_xlabel('Average RSL', fontsize=12)
    axes[1].set_title('Average RSL by Sector', fontsize=14)
    axes[1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig('RSL_Analysis_Charts.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Charts saved as 'RSL_Analysis_Charts.png'")
    
except ImportError:
    print("Matplotlib not available. Skipping graphical visualization.")
    print("To enable charts, run: !pip install matplotlib")

## 12. Final Summary

In [ ]:
print("\n" + "=" * 70)
print("RSL SCREENING COMPLETE")
print("=" * 70)
print(f"\nReport generated: {OUTPUT_FILE}")
print(f"Run timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\nKey Findings:")
print(f"  - Total stocks analyzed: {len(results_df)}")
print(f"  - Top performer: {results_df.iloc[0]['Ticker']} ({results_df.iloc[0]['Company']}) with RSL = {results_df.iloc[0]['RSL']:.4f}")
print(f"  - Weakest stock: {results_df.iloc[-1]['Ticker']} ({results_df.iloc[-1]['Company']}) with RSL = {results_df.iloc[-1]['RSL']:.4f}")
print(f"  - Stocks with bullish momentum (RSL > 1.0): {(results_df['RSL'] > 1.0).sum()}")
print(f"  - Top 25% buy candidates: {len(top_performers)} stocks")
print(f"\nNext Steps:")
print(f"  1. Review the 'Top 25% Buy List' sheet for potential investments")
print(f"  2. Consider sector diversification when building a portfolio")
print(f"  3. Combine RSL with fundamental analysis for better results")
print(f"  4. Re-run this screening periodically (weekly/monthly) to track momentum changes")
print("\n" + "=" * 70)

---

## About the RSL Strategy

### Historical Context
Robert Levy introduced the Relative Strength concept in his 1967 paper "Relative Strength as a Criterion for Investment Selection." His research showed that stocks with high relative strength tend to continue outperforming.

### How to Use This Screening

1. **Buy Candidates**: Focus on the top 25% of stocks by RSL
2. **Avoid**: Stocks in the bottom 25% (weakest momentum)
3. **Rebalance**: Review and rebalance monthly or quarterly
4. **Diversification**: Spread investments across sectors

### Limitations

- Past momentum doesn't guarantee future performance
- Works best in trending markets, may underperform in choppy markets
- Should be combined with other analysis (fundamental, technical)
- Transaction costs can erode returns with frequent rebalancing

### Disclaimer
This tool is for educational and research purposes only. Always do your own research and consider consulting a financial advisor before making investment decisions.

---
*Created with Python | Data from Yahoo Finance & Wikipedia*